# `Safitty` in a nutshell

Safitty is a library for working with deep nested dictionaries and configuration files.
It has several functions for manipulating values and for parsing.

## Creating config

In [79]:
import yaml

# our toy config
config = yaml.safe_load("""
verbose: false

status: 1

paths:
  jsons:
    - first
    - second
    - third:
      - 1
      - 2
      - 3
      - 4
  images: important/path/to/images/

transforms:
  - name: Normalize
    function: ToTensor
    params: null
  -
    name: Padding
    function: Pad
    params:
      fill: 3
      padding_mode: reflect

server:
  address: 127.0.0.1
  password: "qwerty"

reader:
  need_reader: True

  params:
    width: 512
    height: 512
    grayscale: True
""")

config

{'verbose': False,
 'status': 1,
 'paths': {'jsons': ['first', 'second', {'third': [1, 2, 3, 4]}],
  'images': 'important/path/to/images/'},
 'transforms': [{'name': 'Normalize', 'function': 'ToTensor', 'params': None},
  {'name': 'Padding',
   'function': 'Pad',
   'params': {'fill': 3, 'padding_mode': 'reflect'}}],
 'server': {'address': '127.0.0.1', 'password': 'qwerty'},
 'reader': {'need_reader': True,
  'params': {'width': 512, 'height': 512, 'grayscale': True}}}

## Core functions

## `safitty.get`
Simplest function in the library is `get`, which is an analog for nested `dict.get`

In [12]:
import safitty
from pathlib import Path

safitty.get(config, "reader", "params", "grayscale")

True

it is a much more readable equivalent for:

In [14]:
config.get("reader", {}).get("params", {}).get("grayscale")

True

`Get` also works well with nested lists

In [15]:
safitty.get(config, "paths", "jsons", 0)

'first'

In [17]:
safitty.get(config, "paths", "jsons", 2, "third", 3)

4

`get` returns `None` by default if key doesn't exists

In [18]:
safitty.get(config, "paths", "jsons", 2, "third", 5) is None

True

Note, that python's `list` has no safe version for `get`

In [19]:
config["paths"]["jsons"][2]["third"][5]

IndexError: list index out of range

In [21]:
config["paths"]["jsons"][5]["third"][3]

IndexError: list index out of range

an analog may be look like:

In [37]:
KEY1 = 2
KEY2 = 5

result = None
temp = config.get("paths", {}).get("jsons")

if temp is not None and 0 <= KEY1 < len(temp):
    temp = temp[KEY1]
    if temp is not None and "third" in temp:
        temp = temp["third"]
        if temp is not None and 0 <= KEY2 < len(temp):
            result = temp[KEY2]
            del temp

result is None

True

looks terrible

### params

#### `default` – returns default value is anything wrong

In [40]:
safitty.get(config, "paths", "jsons", 2, "third", 5, default="default")

'default'

In [41]:
safitty.get(config, "paths", "jsons", 2, "third", 2, "wrong", "too", "deep", default="default")

'default'

#### `strategy` – strategy for returning values
should be `None` or one of 
- `"missing_key"` – return `default` only if there are error, but not if the final value is None
- `"last_value"` – return last non null value. It doesn't use `default` param
- `"last_container"` – return last non-null container. It doesn't use `default` param

##### missing_key

In [49]:
# in config `params` is set to null
s = safitty.get(config, "transforms", 0, "params")
print(s)

None


In [50]:
s = safitty.get(config, "transforms", 0, "params", default="Oops")
print(s)

Oops


In [51]:
s = safitty.get(config, "transforms", 0, "params", default="oops", strategy="missing_key")
print(s)

None


##### last_value

In [56]:
s = safitty.get(config, "transforms", 0, "function")
print(s)

ToTensor


In [67]:
# too deep
s = safitty.get(config, "transforms", 0, "function", "name", "deep")
print(s)

None


In [68]:
s = safitty.get(config, "transforms", 0, "function", "name", "deep", strategy="last_value")
print(s)

ToTensor


##### last_container

In [66]:
s = safitty.get(config, "transforms", 0, "function", strategy="last_container")
print(s)

{'name': 'Normalize', 'function': 'ToTensor', 'params': None}


In [69]:
s = safitty.get(config, "transforms", 0, "function", "name", "deep", strategy="last_container")
print(s)

{'name': 'Normalize', 'function': 'ToTensor', 'params': None}


#### transform – transform result value with a finction

In [74]:
safitty.get(config, "reader", "params", "height")

512

In [75]:
safitty.get(config, "reader", "params", "height", transform=float)

512.0

In [76]:
safitty.get(config, "reader", "params", "height", transform=lambda x: x * 2)

1024

In [78]:
safitty.get(config, "reader", "params", 0, "height", transform=lambda x: x * 2) # none

#### apply – like `transform` but do `*value` if the result value is a list and `**value` if it's a dict

In [84]:
class Server:
    def __init__(self, address: str, password: str):
        self.address = address
        self.password = password
    def __str__(self) -> str:
        return f"Server at {self.address}. Pass: {self.password}"
    def __repr__(self) -> str:
        return str(self)

In [85]:
safitty.get(config, "server")

{'address': '127.0.0.1', 'password': 'qwerty'}

In [86]:
safitty.get(config, "server", apply=Server)

Server at 127.0.0.1. Pass: qwerty

In [88]:
# still safe
safitty.get(config, "server-less", apply=Server) # not giving any error

#### copy – if true returns a copy 

In [90]:
copied = safitty.get(config, "server", copy=True)
copied

{'address': '127.0.0.1', 'password': 'qwerty'}

In [91]:
copied["password"] = "123456"

In [92]:
safitty.get(config, "server")

{'address': '127.0.0.1', 'password': 'qwerty'}

## `safitty.set`

In [93]:
# tl;dr
safitty.set(config, "new", "additional", "info", value="good")

{'verbose': False,
 'status': 1,
 'paths': {'jsons': ['first', 'second', {'third': [1, 2, 3, 4]}],
  'images': 'important/path/to/images/'},
 'transforms': [{'name': 'Normalize', 'function': 'ToTensor', 'params': None},
  {'name': 'Padding',
   'function': 'Pad',
   'params': {'fill': 3, 'padding_mode': 'reflect'}}],
 'server': {'address': '127.0.0.1', 'password': 'qwerty'},
 'reader': {'need_reader': True,
  'params': {'width': 512, 'height': 512, 'grayscale': True}},
 'new': {'additional': {'info': 'good'}}}

In [94]:
safitty.get(config, "new")

{'additional': {'info': 'good'}}

In [96]:
safitty.get(config, "paths", "jsons", 2, "third", 6) # none

In [97]:
safitty.set(config, "paths", "jsons", 2, "third", 6, value="wow")

{'verbose': False,
 'status': 1,
 'paths': {'jsons': ['first',
   'second',
   {'third': [1, 2, 3, 4, None, None, 'wow']}],
  'images': 'important/path/to/images/'},
 'transforms': [{'name': 'Normalize', 'function': 'ToTensor', 'params': None},
  {'name': 'Padding',
   'function': 'Pad',
   'params': {'fill': 3, 'padding_mode': 'reflect'}}],
 'server': {'address': '127.0.0.1', 'password': 'qwerty'},
 'reader': {'need_reader': True,
  'params': {'width': 512, 'height': 512, 'grayscale': True}},
 'new': {'additional': {'info': 'good'}}}

In [98]:
safitty.get(config, "paths", "jsons", 2, "third")

[1, 2, 3, 4, None, None, 'wow']